# Notebook 71: 連鎖律の解剖 ― 合成関数を分解する

## Chain Rule Decomposition: Breaking Down Composite Functions

---

### このノートブックの位置づけ

**Unit 0.0「ニューラルエンジンの深部」** の第2章として、誤差逆伝播の数学的基盤である **連鎖律（Chain Rule）** を徹底的に理解します。

### 学習目標

1. **連鎖律** を直感的・数式的に理解する
2. 複雑な合成関数を **段階的に分解** して微分する
3. 多変数関数への連鎖律の拡張を理解する
4. ニューラルネットワーク $y = \sigma(Wx + b)$ の各成分の微分を導出する

### 前提知識

- Notebook 70 の内容（微分、偏微分、勾配）

---

## 目次

1. [連鎖律とは：直感的な理解](#1-連鎖律とは直感的な理解)
2. [単純な連鎖：1変数の合成関数](#2-単純な連鎖1変数の合成関数)
3. [多段階の連鎖：深い合成関数](#3-多段階の連鎖深い合成関数)
4. [多変数への拡張：全微分と連鎖律](#4-多変数への拡張全微分と連鎖律)
5. [ニューラルネットワークへの適用](#5-ニューラルネットワークへの適用)
6. [連鎖律の効率性：なぜO(n)なのか](#6-連鎖律の効率性なぜonなのか)
7. [演習問題](#7-演習問題)
8. [まとめと次のステップ](#8-まとめと次のステップ)

In [ ]:
# 環境セットアップ
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import warnings
warnings.filterwarnings('ignore')

# 日本語フォント設定
plt.rcParams['font.family'] = ['Hiragino Sans', 'Arial Unicode MS', 'sans-serif']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11

np.random.seed(42)

print("環境セットアップ完了")

---

## 1. 連鎖律とは：直感的な理解

### 1.1 日常のアナロジー：歯車の連鎖

連鎖律は「**変化が伝播する**」仕組みを表しています。

例えば、自転車のギア比を考えてみましょう：

```
ペダル → 前ギア → チェーン → 後ギア → 車輪
```

- ペダルを1回転させると、前ギア（ギア比2）で2回転
- 後ギア（ギア比0.5）を通って、車輪は1回転
- 全体の変換比 = 2 × 0.5 = 1

**連鎖律も同じ原理です**：各段階の「変化率」を掛け合わせます。

### 1.2 数学的な定義

関数 $y = f(u)$ と $u = g(x)$ が合成されているとき：

$$
y = f(g(x))
$$

この合成関数の $x$ に関する微分は：

$$
\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}
$$

**言葉で言うと**：
- $x$ が変化したとき $u$ がどれだけ変化するか（$\frac{du}{dx}$）
- $u$ が変化したとき $y$ がどれだけ変化するか（$\frac{dy}{du}$）
- これらを掛け合わせると、$x$ → $y$ の変化率が得られる

In [ ]:
# 歯車のアナロジーを可視化
fig, ax = plt.subplots(figsize=(12, 4))
ax.set_xlim(0, 12)
ax.set_ylim(0, 3)
ax.axis('off')

# ノード（歯車）を描画
nodes = [
    (1.5, 1.5, 'x', '入力'),
    (4.5, 1.5, 'u = g(x)', '中間'),
    (7.5, 1.5, 'y = f(u)', '出力'),
]

for x, y, label, sublabel in nodes:
    circle = plt.Circle((x, y), 0.8, fill=True, color='lightblue', 
                         edgecolor='navy', linewidth=2)
    ax.add_patch(circle)
    ax.text(x, y, label, ha='center', va='center', fontsize=12, fontweight='bold')
    ax.text(x, y - 1.2, sublabel, ha='center', va='center', fontsize=10, color='gray')

# 矢印（変化の伝播）を描画
arrow_style = dict(arrowstyle='->', color='darkgreen', lw=2, mutation_scale=15)
ax.annotate('', xy=(3.5, 1.5), xytext=(2.5, 1.5), arrowprops=arrow_style)
ax.annotate('', xy=(6.5, 1.5), xytext=(5.5, 1.5), arrowprops=arrow_style)

# 微分係数のラベル
ax.text(3.0, 2.0, r'$\frac{du}{dx}$', ha='center', va='center', fontsize=14, color='darkgreen')
ax.text(6.0, 2.0, r'$\frac{dy}{du}$', ha='center', va='center', fontsize=14, color='darkgreen')

# 全体の連鎖律
ax.text(10.5, 1.5, r'$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$', 
        ha='center', va='center', fontsize=16, 
        bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='orange'))

ax.set_title('連鎖律：変化が段階的に伝播する', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print("【連鎖律の核心】")
print("各段階の『局所的な変化率』を掛け合わせると、」")
print("『全体の変化率』が得られる")

---

## 2. 単純な連鎖：1変数の合成関数

### 2.1 例題：$(2x + 1)^3$ の微分

関数 $y = (2x + 1)^3$ を考えます。

**分解**：
- 外側の関数: $y = u^3$ where $u = 2x + 1$
- 内側の関数: $u = 2x + 1$

**連鎖律の適用**：
$$
\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx} = 3u^2 \cdot 2 = 6(2x + 1)^2
$$

In [ ]:
# 例題の検証
def y_func(x):
    """y = (2x + 1)^3"""
    return (2*x + 1)**3


def dy_dx_analytical(x):
    """解析的な導関数: dy/dx = 6(2x + 1)^2"""
    return 6 * (2*x + 1)**2


def numerical_diff(f, x, h=1e-7):
    """中心差分による数値微分"""
    return (f(x + h) - f(x - h)) / (2 * h)


# 検証
test_points = [-1.0, 0.0, 0.5, 1.0, 2.0]

print("y = (2x + 1)³ の微分の検証")
print("="*60)
print(f"{'x':>6} | {'解析的 dy/dx':>15} | {'数値微分':>15} | {'誤差':>12}")
print("-"*60)

for x in test_points:
    analytical = dy_dx_analytical(x)
    numerical = numerical_diff(y_func, x)
    error = abs(analytical - numerical)
    print(f"{x:>6.1f} | {analytical:>15.6f} | {numerical:>15.6f} | {error:>12.2e}")

### 2.2 段階ごとの分解を可視化

In [ ]:
# 段階ごとの値と微分を追跡
def trace_chain_rule(x_val):
    """連鎖律の各段階を追跡"""
    print(f"\n【x = {x_val} での連鎖律の追跡】")
    print("="*50)
    
    # Step 1: 内側の関数
    u = 2 * x_val + 1
    du_dx = 2  # u = 2x + 1 の微分
    print(f"\nStep 1: u = 2x + 1")
    print(f"  u = 2 × {x_val} + 1 = {u}")
    print(f"  du/dx = 2")
    
    # Step 2: 外側の関数
    y = u ** 3
    dy_du = 3 * u ** 2  # y = u^3 の微分
    print(f"\nStep 2: y = u³")
    print(f"  y = {u}³ = {y}")
    print(f"  dy/du = 3u² = 3 × {u}² = {dy_du}")
    
    # Step 3: 連鎖律で合成
    dy_dx = dy_du * du_dx
    print(f"\nStep 3: 連鎖律の適用")
    print(f"  dy/dx = dy/du × du/dx")
    print(f"        = {dy_du} × {du_dx}")
    print(f"        = {dy_dx}")
    
    return dy_dx


# 複数の点で追跡
for x in [0, 1, 2]:
    trace_chain_rule(x)
    print()

### 2.3 さらに複雑な例：$\sin(x^2)$

In [ ]:
# y = sin(x²) の微分
# 分解: y = sin(u), u = x²
# dy/dx = dy/du × du/dx = cos(u) × 2x = 2x cos(x²)

def sin_x_squared(x):
    return np.sin(x**2)


def sin_x_squared_derivative(x):
    """解析的導関数: 2x cos(x²)"""
    return 2 * x * np.cos(x**2)


# 可視化
x = np.linspace(-3, 3, 500)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 関数と導関数
axes[0].plot(x, sin_x_squared(x), 'b-', linewidth=2.5, label=r'$y = \sin(x^2)$')
axes[0].plot(x, sin_x_squared_derivative(x), 'r--', linewidth=2, label=r"$y' = 2x\cos(x^2)$")
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('y', fontsize=12)
axes[0].set_title(r'$y = \sin(x^2)$ とその導関数', fontsize=12)
axes[0].legend(fontsize=11)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='k', linewidth=0.5)
axes[0].axvline(x=0, color='k', linewidth=0.5)

# 連鎖律の構造を図示
axes[1].axis('off')
axes[1].set_xlim(0, 10)
axes[1].set_ylim(0, 6)

# ボックスとラベル
boxes = [
    (1, 3, 'x'),
    (4, 3, r'$u = x^2$'),
    (7, 3, r'$y = \sin(u)$'),
]

for bx, by, label in boxes:
    rect = FancyBboxPatch((bx-0.8, by-0.5), 1.6, 1, boxstyle="round,pad=0.1",
                          facecolor='lightblue', edgecolor='navy', linewidth=2)
    axes[1].add_patch(rect)
    axes[1].text(bx, by, label, ha='center', va='center', fontsize=14)

# 矢印
axes[1].annotate('', xy=(3, 3), xytext=(2, 3),
                 arrowprops=dict(arrowstyle='->', color='darkgreen', lw=2))
axes[1].annotate('', xy=(6, 3), xytext=(5, 3),
                 arrowprops=dict(arrowstyle='->', color='darkgreen', lw=2))

# 微分のラベル
axes[1].text(2.5, 3.8, r'$\frac{du}{dx} = 2x$', ha='center', fontsize=12, color='darkgreen')
axes[1].text(5.5, 3.8, r'$\frac{dy}{du} = \cos(u)$', ha='center', fontsize=12, color='darkgreen')

# 結果
axes[1].text(5, 1.2, r'$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx} = \cos(x^2) \cdot 2x = 2x\cos(x^2)$',
             ha='center', fontsize=14, 
             bbox=dict(boxstyle='round', facecolor='lightyellow', edgecolor='orange'))

axes[1].set_title('連鎖律の適用プロセス', fontsize=12)

plt.tight_layout()
plt.show()

---

## 3. 多段階の連鎖：深い合成関数

### 3.1 3層以上の合成

ニューラルネットワークは何層にも重なった合成関数です。3層の例を考えます：

$$
y = f(g(h(x)))
$$

連鎖律を繰り返し適用：

$$
\frac{dy}{dx} = \frac{dy}{dv} \cdot \frac{dv}{du} \cdot \frac{du}{dx}
$$

where $u = h(x)$, $v = g(u)$, $y = f(v)$

In [ ]:
# 3層の合成関数の例
# y = exp(sin(x²))
# 分解: h(x) = x², g(u) = sin(u), f(v) = exp(v)

def three_layer_composite(x):
    """y = exp(sin(x²))"""
    return np.exp(np.sin(x**2))


def three_layer_derivative(x):
    """
    dy/dx = dy/dv × dv/du × du/dx
          = exp(v) × cos(u) × 2x
          = exp(sin(x²)) × cos(x²) × 2x
    """
    u = x**2
    v = np.sin(u)
    return np.exp(v) * np.cos(u) * 2 * x


def trace_three_layer(x_val):
    """3層の連鎖律を追跡"""
    print(f"\n【x = {x_val} での3層連鎖律】")
    print("="*60)
    
    # 順伝播（Forward）
    print("\n[順伝播: 値を前へ送る]")
    u = x_val ** 2
    print(f"  x = {x_val}")
    print(f"  u = h(x) = x² = {u:.4f}")
    
    v = np.sin(u)
    print(f"  v = g(u) = sin(u) = sin({u:.4f}) = {v:.4f}")
    
    y = np.exp(v)
    print(f"  y = f(v) = exp(v) = exp({v:.4f}) = {y:.4f}")
    
    # 逆伝播（Backward）- 微分を逆向きに計算
    print("\n[逆伝播: 微分を後ろへ送る]")
    
    dy_dv = np.exp(v)
    print(f"  dy/dv = exp(v) = {dy_dv:.4f}")
    
    dv_du = np.cos(u)
    print(f"  dv/du = cos(u) = cos({u:.4f}) = {dv_du:.4f}")
    
    du_dx = 2 * x_val
    print(f"  du/dx = 2x = {du_dx:.4f}")
    
    # 連鎖律で合成
    dy_dx = dy_dv * dv_du * du_dx
    print(f"\n[連鎖律の適用]")
    print(f"  dy/dx = dy/dv × dv/du × du/dx")
    print(f"        = {dy_dv:.4f} × {dv_du:.4f} × {du_dx:.4f}")
    print(f"        = {dy_dx:.4f}")
    
    # 数値微分で検証
    numerical = numerical_diff(three_layer_composite, x_val)
    print(f"\n[検証] 数値微分: {numerical:.4f}")
    print(f"       誤差: {abs(dy_dx - numerical):.2e}")


# 追跡実行
trace_three_layer(1.0)

### 3.2 計算の流れを図解

In [ ]:
# 3層の計算グラフを可視化
fig, axes = plt.subplots(2, 1, figsize=(14, 8))

# 上: 順伝播（Forward Pass）
ax = axes[0]
ax.set_xlim(0, 14)
ax.set_ylim(0, 3)
ax.axis('off')

# ノード
nodes_forward = [
    (1, 1.5, 'x', 'x=1'),
    (4, 1.5, '$u = x^2$', 'u=1'),
    (7, 1.5, '$v = \\sin(u)$', 'v=0.84'),
    (10, 1.5, '$y = e^v$', 'y=2.32'),
]

for nx, ny, label, value in nodes_forward:
    rect = FancyBboxPatch((nx-1, ny-0.5), 2, 1, boxstyle="round,pad=0.1",
                          facecolor='lightgreen', edgecolor='darkgreen', linewidth=2)
    ax.add_patch(rect)
    ax.text(nx, ny + 0.1, label, ha='center', va='center', fontsize=11)
    ax.text(nx, ny - 0.3, value, ha='center', va='center', fontsize=10, color='blue')

# 矢印（順方向）
for i in range(3):
    ax.annotate('', xy=(nodes_forward[i+1][0]-1.1, 1.5), 
                xytext=(nodes_forward[i][0]+1.1, 1.5),
                arrowprops=dict(arrowstyle='->', color='green', lw=2))

ax.set_title('順伝播（Forward Pass）: 値を前へ送る →', fontsize=14, color='darkgreen', pad=10)
ax.text(12.5, 1.5, '値の\n計算', ha='center', va='center', fontsize=11,
        bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.5))

# 下: 逆伝播（Backward Pass）
ax = axes[1]
ax.set_xlim(0, 14)
ax.set_ylim(0, 3)
ax.axis('off')

# ノード（微分値付き）
nodes_backward = [
    (1, 1.5, 'x', '$\\frac{du}{dx}=2$'),
    (4, 1.5, 'u', '$\\frac{dv}{du}=0.54$'),
    (7, 1.5, 'v', '$\\frac{dy}{dv}=2.32$'),
    (10, 1.5, 'y', '(出力)'),
]

for nx, ny, label, deriv in nodes_backward:
    rect = FancyBboxPatch((nx-1, ny-0.5), 2, 1, boxstyle="round,pad=0.1",
                          facecolor='lightyellow', edgecolor='orange', linewidth=2)
    ax.add_patch(rect)
    ax.text(nx, ny + 0.1, label, ha='center', va='center', fontsize=12, fontweight='bold')
    ax.text(nx, ny - 0.3, deriv, ha='center', va='center', fontsize=9, color='red')

# 矢印（逆方向）
for i in range(3):
    ax.annotate('', xy=(nodes_backward[i][0]+1.1, 1.5),
                xytext=(nodes_backward[i+1][0]-1.1, 1.5),
                arrowprops=dict(arrowstyle='->', color='red', lw=2))

ax.set_title('逆伝播（Backward Pass）: 勾配を後ろへ送る ←', fontsize=14, color='darkred', pad=10)

# 最終結果
ax.text(12.5, 1.5, 'dy/dx\n=2.51', ha='center', va='center', fontsize=11,
        bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

plt.tight_layout()
plt.show()

print("【重要な洞察】")
print("・順伝播: x → u → v → y の順に『値』を計算")
print("・逆伝播: y → v → u → x の順に『微分』を伝播")
print("・これが誤差逆伝播法（Backpropagation）の本質")

---

## 4. 多変数への拡張：全微分と連鎖律

### 4.1 複数の入力を持つ関数

ニューラルネットワークでは、1つの出力が **複数の入力** から計算されます。

例えば、$z = f(x, y)$ で $x = x(t)$, $y = y(t)$ のとき：

$$
\frac{dz}{dt} = \frac{\partial z}{\partial x} \cdot \frac{dx}{dt} + \frac{\partial z}{\partial y} \cdot \frac{dy}{dt}
$$

これは **全微分の連鎖律** と呼ばれます。

### 4.2 加算ノードと乗算ノードの分岐

In [ ]:
# 分岐と合流を持つ計算グラフの例
# z = x*y + sin(x)

def branching_example(x, y):
    """z = x*y + sin(x)"""
    return x * y + np.sin(x)


def trace_branching(x_val, y_val):
    """
    z = x*y + sin(x) の勾配を追跡
    
    計算グラフ:
         x ──┬── (*) ──┐
             │         │
         y ──┘         (+) ── z
             │         │
         x ──(sin)────┘
    
    ∂z/∂x = ∂z/∂(x*y) × ∂(x*y)/∂x + ∂z/∂(sin(x)) × ∂(sin(x))/∂x
          = 1 × y + 1 × cos(x)
          = y + cos(x)
    
    ∂z/∂y = ∂z/∂(x*y) × ∂(x*y)/∂y
          = 1 × x
          = x
    """
    print(f"\n【分岐のある計算グラフ】")
    print(f"z = x*y + sin(x) at (x, y) = ({x_val}, {y_val})")
    print("="*50)
    
    # 順伝播
    print("\n[順伝播]")
    term1 = x_val * y_val
    print(f"  x*y = {x_val} × {y_val} = {term1}")
    term2 = np.sin(x_val)
    print(f"  sin(x) = sin({x_val}) = {term2:.4f}")
    z = term1 + term2
    print(f"  z = {term1} + {term2:.4f} = {z:.4f}")
    
    # 逆伝播（勾配）
    print("\n[逆伝播]")
    print("  dz/dz = 1（出発点）")
    print("")
    print("  加算ノード (+) で分岐:")
    print("    → x*y への勾配 = 1")
    print("    → sin(x) への勾配 = 1")
    print("")
    print("  乗算ノード (x*y) で:")
    print(f"    → x への勾配 = y = {y_val}")
    print(f"    → y への勾配 = x = {x_val}")
    print("")
    print("  sin ノード で:")
    print(f"    → x への勾配 = cos(x) = cos({x_val}) = {np.cos(x_val):.4f}")
    print("")
    print("  x は2つの経路から勾配を受け取る（合流）:")
    dz_dx = y_val + np.cos(x_val)
    dz_dy = x_val
    print(f"    ∂z/∂x = y + cos(x) = {y_val} + {np.cos(x_val):.4f} = {dz_dx:.4f}")
    print(f"    ∂z/∂y = x = {dz_dy}")
    
    return dz_dx, dz_dy


# 実行と検証
x, y = 2.0, 3.0
dz_dx, dz_dy = trace_branching(x, y)

# 数値微分で検証
h = 1e-7
numerical_dz_dx = (branching_example(x + h, y) - branching_example(x - h, y)) / (2 * h)
numerical_dz_dy = (branching_example(x, y + h) - branching_example(x, y - h)) / (2 * h)

print(f"\n[数値微分による検証]")
print(f"  ∂z/∂x: 解析的 = {dz_dx:.6f}, 数値 = {numerical_dz_dx:.6f}")
print(f"  ∂z/∂y: 解析的 = {dz_dy:.6f}, 数値 = {numerical_dz_dy:.6f}")

### 4.3 分岐と合流の視覚化

In [ ]:
# 分岐・合流を持つ計算グラフを図示
fig, ax = plt.subplots(figsize=(12, 6))
ax.set_xlim(0, 12)
ax.set_ylim(0, 6)
ax.axis('off')

# ノードの定義
nodes = {
    'x': (1, 4),
    'y': (1, 2),
    'mul': (4, 3),
    'sin': (4, 5),
    'add': (7, 4),
    'z': (10, 4),
}

# ノードを描画
node_labels = {
    'x': ('x', 'lightblue'),
    'y': ('y', 'lightblue'),
    'mul': ('×', 'lightgreen'),
    'sin': ('sin', 'lightyellow'),
    'add': ('+', 'lightgreen'),
    'z': ('z', 'lightcoral'),
}

for name, (nx, ny) in nodes.items():
    label, color = node_labels[name]
    circle = plt.Circle((nx, ny), 0.5, facecolor=color, edgecolor='black', linewidth=2)
    ax.add_patch(circle)
    ax.text(nx, ny, label, ha='center', va='center', fontsize=14, fontweight='bold')

# エッジ（順伝播）
edges = [
    ('x', 'mul', '', 'black'),
    ('y', 'mul', '', 'black'),
    ('x', 'sin', '', 'black'),
    ('mul', 'add', '', 'black'),
    ('sin', 'add', '', 'black'),
    ('add', 'z', '', 'black'),
]

for start, end, label, color in edges:
    sx, sy = nodes[start]
    ex, ey = nodes[end]
    ax.annotate('', xy=(ex - 0.5, ey), xytext=(sx + 0.5, sy),
                arrowprops=dict(arrowstyle='->', color=color, lw=1.5))

# 勾配のラベル（逆伝播の説明）
gradient_labels = [
    (2.5, 3.8, r'$\frac{\partial(xy)}{\partial x} = y$', 'blue'),
    (2.5, 2.2, r'$\frac{\partial(xy)}{\partial y} = x$', 'blue'),
    (2.5, 4.8, r'$\frac{\partial\sin(x)}{\partial x} = \cos(x)$', 'blue'),
    (5.5, 3.8, '1', 'red'),
    (5.5, 4.8, '1', 'red'),
    (8.5, 4.2, '1', 'red'),
]

for lx, ly, text, color in gradient_labels:
    ax.text(lx, ly, text, ha='center', va='center', fontsize=10, color=color)

# 分岐点の強調
ax.annotate('分岐\n(x が2経路へ)', xy=(1, 4), xytext=(0, 5.5),
            fontsize=10, ha='center',
            arrowprops=dict(arrowstyle='->', color='purple', lw=1))

# 合流点の強調
ax.annotate('合流\n(勾配が加算される)', xy=(7, 4), xytext=(7, 1.5),
            fontsize=10, ha='center',
            arrowprops=dict(arrowstyle='->', color='purple', lw=1))

ax.set_title('分岐と合流を持つ計算グラフ: $z = xy + \\sin(x)$', fontsize=14)

plt.tight_layout()
plt.show()

print("【重要なルール】")
print("・分岐: 同じ変数が複数の演算に使われる")
print("・合流: 逆伝播では、複数経路からの勾配を『加算』する")
print("")
print("∂z/∂x = (経路1: xy経由) + (経路2: sin経由)")
print("       = y + cos(x)")

---

## 5. ニューラルネットワークへの適用

### 5.1 単純なニューロン $y = \sigma(Wx + b)$

最も基本的なニューラルネットワークの単位を微分してみましょう。

**パラメータ**:
- 入力: $x$
- 重み: $W$
- バイアス: $b$
- 活性化関数: $\sigma$ (シグモイド)

**計算グラフ**:
$$
x \xrightarrow{W \cdot} z_1 = Wx \xrightarrow{+ b} z_2 = Wx + b \xrightarrow{\sigma} y = \sigma(Wx + b)
$$

In [ ]:
# y = σ(Wx + b) の完全な順伝播・逆伝播

def sigmoid(z):
    return 1 / (1 + np.exp(-z))


def sigmoid_derivative(z):
    s = sigmoid(z)
    return s * (1 - s)


def neuron_forward_backward(x, W, b, verbose=True):
    """
    y = σ(Wx + b) の順伝播と逆伝播
    
    Returns:
        y: 出力
        grads: 各パラメータの勾配 {dy_dx, dy_dW, dy_db}
    """
    if verbose:
        print(f"\n【単一ニューロン y = σ(Wx + b)】")
        print(f"入力: x = {x}, W = {W}, b = {b}")
        print("="*60)
    
    # ========== 順伝播 ==========
    if verbose:
        print("\n[順伝播]")
    
    # Step 1: z1 = W * x
    z1 = W * x
    if verbose:
        print(f"  z1 = W × x = {W} × {x} = {z1}")
    
    # Step 2: z2 = z1 + b
    z2 = z1 + b
    if verbose:
        print(f"  z2 = z1 + b = {z1} + {b} = {z2}")
    
    # Step 3: y = σ(z2)
    y = sigmoid(z2)
    if verbose:
        print(f"  y = σ(z2) = σ({z2}) = {y:.6f}")
    
    # ========== 逆伝播 ==========
    if verbose:
        print("\n[逆伝播]")
        print("  dy/dy = 1（出発点）")
    
    # Step 3の逆: dy/dz2 = σ'(z2) = σ(z2)(1 - σ(z2))
    dy_dz2 = sigmoid_derivative(z2)
    if verbose:
        print(f"\n  σノード:")
        print(f"    dy/dz2 = σ'(z2) = σ(z2)(1-σ(z2)) = {y:.6f} × {1-y:.6f} = {dy_dz2:.6f}")
    
    # Step 2の逆: dz2/dz1 = 1, dz2/db = 1
    dz2_dz1 = 1
    dz2_db = 1
    if verbose:
        print(f"\n  加算ノード (+b):")
        print(f"    dz2/dz1 = 1")
        print(f"    dz2/db = 1")
    
    # 連鎖律: dy/dz1, dy/db
    dy_dz1 = dy_dz2 * dz2_dz1
    dy_db = dy_dz2 * dz2_db
    if verbose:
        print(f"    dy/dz1 = dy/dz2 × dz2/dz1 = {dy_dz2:.6f} × 1 = {dy_dz1:.6f}")
        print(f"    dy/db = dy/dz2 × dz2/db = {dy_dz2:.6f} × 1 = {dy_db:.6f}")
    
    # Step 1の逆: dz1/dW = x, dz1/dx = W
    dz1_dW = x
    dz1_dx = W
    if verbose:
        print(f"\n  乗算ノード (W×x):")
        print(f"    dz1/dW = x = {x}")
        print(f"    dz1/dx = W = {W}")
    
    # 連鎖律: dy/dW, dy/dx
    dy_dW = dy_dz1 * dz1_dW
    dy_dx = dy_dz1 * dz1_dx
    if verbose:
        print(f"    dy/dW = dy/dz1 × dz1/dW = {dy_dz1:.6f} × {x} = {dy_dW:.6f}")
        print(f"    dy/dx = dy/dz1 × dz1/dx = {dy_dz1:.6f} × {W} = {dy_dx:.6f}")
    
    # 結果のまとめ
    if verbose:
        print("\n[最終結果]")
        print(f"  ∂y/∂x = {dy_dx:.6f}")
        print(f"  ∂y/∂W = {dy_dW:.6f}")
        print(f"  ∂y/∂b = {dy_db:.6f}")
    
    return y, {'dy_dx': dy_dx, 'dy_dW': dy_dW, 'dy_db': dy_db}


# 実行
x, W, b = 2.0, 0.5, -0.3
y, grads = neuron_forward_backward(x, W, b)

In [ ]:
# 数値微分で検証
def neuron_output(x, W, b):
    return sigmoid(W * x + b)


h = 1e-7
x, W, b = 2.0, 0.5, -0.3

# 数値微分
numerical_dy_dx = (neuron_output(x + h, W, b) - neuron_output(x - h, W, b)) / (2 * h)
numerical_dy_dW = (neuron_output(x, W + h, b) - neuron_output(x, W - h, b)) / (2 * h)
numerical_dy_db = (neuron_output(x, W, b + h) - neuron_output(x, W, b - h)) / (2 * h)

# 比較
y, grads = neuron_forward_backward(x, W, b, verbose=False)

print("【勾配チェック: y = σ(Wx + b)】")
print("="*60)
print(f"{'パラメータ':>10} | {'解析的勾配':>12} | {'数値微分':>12} | {'誤差':>10}")
print("-"*60)
print(f"{'∂y/∂x':>10} | {grads['dy_dx']:>12.8f} | {numerical_dy_dx:>12.8f} | {abs(grads['dy_dx'] - numerical_dy_dx):>10.2e}")
print(f"{'∂y/∂W':>10} | {grads['dy_dW']:>12.8f} | {numerical_dy_dW:>12.8f} | {abs(grads['dy_dW'] - numerical_dy_dW):>10.2e}")
print(f"{'∂y/∂b':>10} | {grads['dy_db']:>12.8f} | {numerical_dy_db:>12.8f} | {abs(grads['dy_db'] - numerical_dy_db):>10.2e}")

### 5.2 損失関数を含めた完全な逆伝播

In [ ]:
# 損失関数を追加: L = (y - t)² / 2
# t: 教師信号（正解）

def full_forward_backward(x, W, b, t, verbose=True):
    """
    完全な順伝播・逆伝播
    
    順伝播: x → z = Wx + b → y = σ(z) → L = (y - t)² / 2
    逆伝播: dL/dW, dL/db を計算
    """
    if verbose:
        print(f"\n【完全な順伝播・逆伝播】")
        print(f"入力: x = {x}, W = {W}, b = {b}, t(正解) = {t}")
        print("="*60)
    
    # ========== 順伝播 ==========
    z = W * x + b
    y = sigmoid(z)
    L = 0.5 * (y - t) ** 2
    
    if verbose:
        print("\n[順伝播]")
        print(f"  z = Wx + b = {W} × {x} + {b} = {z}")
        print(f"  y = σ(z) = {y:.6f}")
        print(f"  L = (y - t)²/2 = ({y:.6f} - {t})²/2 = {L:.6f}")
    
    # ========== 逆伝播 ==========
    if verbose:
        print("\n[逆伝播]")
    
    # dL/dy = y - t
    dL_dy = y - t
    if verbose:
        print(f"  dL/dy = y - t = {y:.6f} - {t} = {dL_dy:.6f}")
    
    # dy/dz = σ'(z)
    dy_dz = sigmoid_derivative(z)
    if verbose:
        print(f"  dy/dz = σ'(z) = {dy_dz:.6f}")
    
    # dL/dz = dL/dy × dy/dz (これが δ: デルタ と呼ばれる)
    dL_dz = dL_dy * dy_dz
    if verbose:
        print(f"  dL/dz = dL/dy × dy/dz = {dL_dy:.6f} × {dy_dz:.6f} = {dL_dz:.6f}")
        print(f"  (この dL/dz が『誤差信号 δ』と呼ばれる)")
    
    # dL/dW = dL/dz × dz/dW = δ × x
    dL_dW = dL_dz * x
    if verbose:
        print(f"\n  dL/dW = δ × x = {dL_dz:.6f} × {x} = {dL_dW:.6f}")
    
    # dL/db = dL/dz × dz/db = δ × 1
    dL_db = dL_dz * 1
    if verbose:
        print(f"  dL/db = δ × 1 = {dL_db:.6f}")
    
    # dL/dx = dL/dz × dz/dx = δ × W（次の層への逆伝播用）
    dL_dx = dL_dz * W
    if verbose:
        print(f"  dL/dx = δ × W = {dL_dz:.6f} × {W} = {dL_dx:.6f}（次の層へ伝播）")
    
    if verbose:
        print("\n[最終結果: パラメータ更新に使う勾配]")
        print(f"  ∂L/∂W = {dL_dW:.6f}")
        print(f"  ∂L/∂b = {dL_db:.6f}")
    
    return L, {'dL_dW': dL_dW, 'dL_db': dL_db, 'dL_dx': dL_dx}


# 実行
x, W, b, t = 2.0, 0.5, -0.3, 1.0
L, grads = full_forward_backward(x, W, b, t)

---

## 6. 連鎖律の効率性：なぜO(n)なのか

### 6.1 計算量の分析

$n$ 層のネットワークの全パラメータに対する勾配を計算する場合：

**ナイーブな方法（数値微分）**:
- 各パラメータごとに順伝播を2回 → $O(n \times \text{パラメータ数})$
- 数百万パラメータのモデルでは非現実的

**連鎖律（逆伝播）**:
- 順伝播1回 + 逆伝播1回 → $O(n)$
- パラメータ数に依存しない！

In [ ]:
# 計算量の比較を可視化
import time

def benchmark_gradient_computation(n_params_list):
    """勾配計算の計算時間を比較"""
    numerical_times = []
    analytical_times = []
    
    for n_params in n_params_list:
        # ダミーの重み
        W = np.random.randn(n_params)
        x = np.random.randn()
        
        # 簡単な関数: y = Σ W_i * x
        def forward(W, x):
            return np.sum(W * x)
        
        # 数値微分（各パラメータに対して）
        start = time.time()
        h = 1e-7
        numerical_grads = np.zeros(n_params)
        for i in range(n_params):
            W_plus = W.copy()
            W_plus[i] += h
            W_minus = W.copy()
            W_minus[i] -= h
            numerical_grads[i] = (forward(W_plus, x) - forward(W_minus, x)) / (2 * h)
        numerical_times.append(time.time() - start)
        
        # 解析的勾配（連鎖律：一度に全部）
        start = time.time()
        analytical_grads = np.full(n_params, x)  # dy/dW_i = x
        analytical_times.append(time.time() - start)
    
    return numerical_times, analytical_times


# ベンチマーク実行
n_params_list = [10, 50, 100, 500, 1000, 5000]
numerical_times, analytical_times = benchmark_gradient_computation(n_params_list)

# 可視化
fig, ax = plt.subplots(figsize=(10, 6))

ax.plot(n_params_list, numerical_times, 'ro-', label='数値微分 O(n × パラメータ数)', linewidth=2, markersize=8)
ax.plot(n_params_list, analytical_times, 'bs-', label='連鎖律（逆伝播） O(n)', linewidth=2, markersize=8)

ax.set_xlabel('パラメータ数', fontsize=12)
ax.set_ylabel('計算時間 (秒)', fontsize=12)
ax.set_title('勾配計算の計算量比較', fontsize=14)
ax.legend(fontsize=11)
ax.grid(True, alpha=0.3)
ax.set_yscale('log')

plt.tight_layout()
plt.show()

print("【計算効率の違い】")
print(f"パラメータ数 5000 の場合:")
print(f"  数値微分: {numerical_times[-1]:.4f} 秒")
print(f"  連鎖律:   {analytical_times[-1]:.6f} 秒")
print(f"  速度比:   {numerical_times[-1] / analytical_times[-1]:.0f} 倍")

---

## 7. 演習問題

### 演習 7.1: 連鎖律の手計算

次の合成関数の微分を、連鎖律を使って手計算で求めてください。

$$
y = \ln(1 + e^{2x})
$$

ヒント: softplus関数と呼ばれる活性化関数です。

In [ ]:
# 演習 7.1: 解答欄

def softplus_2x(x):
    """y = ln(1 + e^(2x))"""
    return np.log(1 + np.exp(2*x))


def softplus_2x_derivative(x):
    """
    TODO: 連鎖律で導出した導関数を実装
    
    ヒント:
    - u = 2x, v = e^u, w = 1 + v, y = ln(w)
    - dy/dx = dy/dw × dw/dv × dv/du × du/dx
    """
    pass


# テストコード（実装後にコメントを外して実行）
# test_points = [-2.0, -1.0, 0.0, 1.0, 2.0]
# print("softplus(2x) = ln(1 + e^(2x)) の勾配チェック")
# for x in test_points:
#     analytical = softplus_2x_derivative(x)
#     numerical = numerical_diff(softplus_2x, x)
#     print(f"x = {x:5.1f}: 解析的 = {analytical:.6f}, 数値 = {numerical:.6f}")

### 演習 7.2: 2入力ニューロン

入力が2つの場合のニューロン $y = \sigma(w_1 x_1 + w_2 x_2 + b)$ について、$\frac{\partial y}{\partial w_1}$ と $\frac{\partial y}{\partial w_2}$ を導出し、実装してください。

In [ ]:
# 演習 7.2: 解答欄

def two_input_neuron_forward(x1, x2, w1, w2, b):
    """y = σ(w1*x1 + w2*x2 + b)"""
    z = w1 * x1 + w2 * x2 + b
    y = sigmoid(z)
    return y


def two_input_neuron_backward(x1, x2, w1, w2, b):
    """
    TODO: 逆伝播を実装
    
    Returns:
        dy_dw1, dy_dw2, dy_db
    """
    pass


# テストコード（実装後にコメントを外して実行）
# x1, x2, w1, w2, b = 1.0, 2.0, 0.5, -0.3, 0.1
# dy_dw1, dy_dw2, dy_db = two_input_neuron_backward(x1, x2, w1, w2, b)
# 
# # 数値微分で検証
# h = 1e-7
# num_dy_dw1 = (two_input_neuron_forward(x1, x2, w1+h, w2, b) - 
#               two_input_neuron_forward(x1, x2, w1-h, w2, b)) / (2*h)
# print(f"dy/dw1: 解析的 = {dy_dw1:.6f}, 数値 = {num_dy_dw1:.6f}")

### 演習 7.3: ReLU活性化関数

シグモイドの代わりにReLUを使った場合、$y = \text{ReLU}(Wx + b)$ の逆伝播を実装してください。

In [ ]:
# 演習 7.3: 解答欄

def relu(z):
    return np.maximum(0, z)


def relu_derivative(z):
    """
    TODO: ReLUの導関数を実装
    ヒント: z > 0 なら 1, そうでなければ 0
    """
    pass


def relu_neuron_backward(x, W, b):
    """
    y = ReLU(Wx + b) の逆伝播
    
    TODO: dy/dW, dy/db, dy/dx を計算
    """
    pass


# テストコード

---

## 8. まとめと次のステップ

### このノートブックで学んだこと

1. **連鎖律の本質**: 合成関数の微分 = 局所的な微分の積
   $$\frac{dy}{dx} = \frac{dy}{du} \cdot \frac{du}{dx}$$

2. **多段階の連鎖**: 深い合成関数でも同じ原理を繰り返し適用

3. **分岐と合流**:
   - 分岐: 1つの変数が複数の経路に影響
   - 合流: 複数の経路からの勾配を **加算**

4. **ニューラルネットワークへの適用**:
   - $y = \sigma(Wx + b)$ の逆伝播
   - 誤差信号 $\delta = \frac{\partial L}{\partial z}$ の概念

5. **計算効率**: 連鎖律により $O(n)$ で全パラメータの勾配を計算可能

### 次のノートブック（72: 計算グラフ）への橋渡し

連鎖律の計算を **自動化** するために、計算を **グラフ構造** として表現します：

- 各演算をノード（Node）として表現
- 順伝播: データがグラフを **前向き** に流れる
- 逆伝播: 勾配がグラフを **後ろ向き** に流れる

次のノートブックでは、この計算グラフをPythonクラスで実装します。

---

## 付録: 連鎖律の公式集

### 基本形

| 形式 | 連鎖律 |
|------|--------|
| $y = f(g(x))$ | $\frac{dy}{dx} = f'(g(x)) \cdot g'(x)$ |
| $y = f(g(h(x)))$ | $\frac{dy}{dx} = f'(g(h(x))) \cdot g'(h(x)) \cdot h'(x)$ |

### 多変数への拡張

| 状況 | 連鎖律 |
|------|--------|
| $z = f(x, y)$, $x = x(t)$, $y = y(t)$ | $\frac{dz}{dt} = \frac{\partial z}{\partial x}\frac{dx}{dt} + \frac{\partial z}{\partial y}\frac{dy}{dt}$ |
| 分岐（$x$ が複数経路に影響） | 各経路からの勾配を **加算** |

### ニューラルネットワークでよく使う形

| 演算 | 順伝播 | 逆伝播 |
|------|--------|--------|
| 加算 $z = x + y$ | そのまま | $\frac{\partial L}{\partial x} = \frac{\partial L}{\partial z}$, $\frac{\partial L}{\partial y} = \frac{\partial L}{\partial z}$ |
| 乗算 $z = x \cdot y$ | そのまま | $\frac{\partial L}{\partial x} = y \cdot \frac{\partial L}{\partial z}$, $\frac{\partial L}{\partial y} = x \cdot \frac{\partial L}{\partial z}$ |
| シグモイド $y = \sigma(x)$ | $\sigma(x)$ | $\frac{\partial L}{\partial x} = \sigma(x)(1-\sigma(x)) \cdot \frac{\partial L}{\partial y}$ |
| ReLU $y = \max(0, x)$ | $\max(0, x)$ | $\frac{\partial L}{\partial x} = \mathbf{1}_{x > 0} \cdot \frac{\partial L}{\partial y}$ |